In [1]:
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

# Préstamos

In [2]:
%%time

dtypes = {
    'TITULO': str,
    'AUTOR': str,
    'SUCURSAL': str,
    'COD_BARRAS_LIBRO': str,
    'DEWEY': str,
    'DESC_LOCALIZACION_EJEMPLAR': str,
    'FECHA_PUB': str,
    'PAIS_EJEMPLAR': str,
    'OCUPACION': str,
    'NIVEL_EDUCACION': str,
    'AREA_ESTUDIO': str,
    'SUCURSAL_PRESTAMO': str,
    'SUCURSAL_DEVOLUCION': str,
    'SUCURSAL_DEL_LECTOR': str,
    'COD_BARRAS_LECTOR': str,
    'CATEGORÍA': str,
    'FECHA_PRESTAMO': str,
    'FECHA_REAL_DEVOLUCION': str,
    'DATE_PRESTAMO': pd.tslib.Timestamp,
    'DATE_DEVOLUCION': pd.tslib.Timestamp,
}

prestamos_df = pd.read_csv('prestamos_totales.csv', header=0, dtype=dtypes, parse_dates=['DATE_PRESTAMO', 'DATE_DEVOLUCION'])
prestamos_df = prestamos_df.fillna('')

CPU times: user 19.7 s, sys: 1.4 s, total: 21.1 s
Wall time: 21.8 s


In [ ]:
prestamos_df

# Autores

In [3]:
%%time

dtypes = {
    'TITULO': str,
    'AUTOR': str,
    'PAIS_EJEMPLAR': str,
    'PAIS_AUTOR': str,
}

autores_df = pd.read_csv('autores5.csv', header=0, dtype=dtypes)
autores_df = autores_df.fillna('')

CPU times: user 167 ms, sys: 0 ns, total: 167 ms
Wall time: 189 ms


In [21]:
%%time

dtypes = {
    'TITULO': str,
    'AUTOR': str,
    'PAIS_EJEMPLAR': str,
    'PAIS_AUTOR': str,
}

qqq = pd.read_csv('autores.csv', header=0, dtype=dtypes)
qqq = qqq.fillna('')

CPU times: user 6.4 s, sys: 330 ms, total: 6.73 s
Wall time: 7.16 s


In [22]:
qqq.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR
0,Un bicho extraño,Spain,Mon Daporta,DESCONOCIDO
1,Un bicho extraño,Spain,Mon Daporta,DESCONOCIDO
2,Quisiera tener ...,Mexico,Giovanna Zoboli,DESCONOCIDO
3,Johannes Gutenberg,Spain,Lluís Borràs Perelló,DESCONOCIDO
4,La fantástica leyenda de: la princesa y el dragón,Spain,Sonia Alins,DESCONOCIDO


In [23]:
qqq['AUTOR'].nunique()

145132

In [29]:
def norm(s):
    return s \
        .replace('á', 'a') \
        .replace('à', 'a') \
        .replace('é', 'e') \
        .replace('è', 'e') \
        .replace('í', 'i') \
        .replace('ì', 'i') \
        .replace('ó', 'o') \
        .replace('ò', 'o') \
        .replace('ú', 'u') \
        .replace('ù', 'u')

In [31]:
norm('asdádssùd')

'asdadssud'

In [32]:
ls = [norm(x) for x in qqq['AUTOR'].unique()]

In [33]:
len(ls)

145132

In [34]:
ss = set(ls)

In [35]:
len(ss)

144083

In [4]:
autores_df.shape

(76697, 4)

In [5]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR
0,Adivina en qué pais ...,Spain,Martina Badstuber,Germany
1,¡No quiero hacer pipí en el orinal!,Spain,Roser Rius,Mexico
2,Humo,Spain,Antón Fortes Torres,Spain
3,Galleta para perros,Spain,Helen Cooper,United Kingdom
4,Autobio,Spain,Cyril Pedrosa,France


# Rutas

In [7]:
autores_df['PAIS_AUTOR'].nunique()

245

In [19]:
autores_df['PAIS_AUTOR'].value_counts().head()

United States     11803
Spain              8264
France             7498
United Kingdom     6673
Germany            4665
Name: PAIS_AUTOR, dtype: int64

In [6]:
autores_df['PAIS_EJEMPLAR'].nunique()

121

In [20]:
autores_df['PAIS_EJEMPLAR'].value_counts().head()

Spain            37448
Colombia          9876
Argentina         7286
Mexico            6792
United States     3917
Name: PAIS_EJEMPLAR, dtype: int64

In [12]:
%%time

pairs = []
for i, x in autores_df.iterrows():
    pais_autor = x['PAIS_AUTOR']
    pais_ejemplar = x['PAIS_EJEMPLAR']
    if pais_autor != pais_ejemplar:
        pairs.append((pais_autor, pais_ejemplar))

In [14]:
pairs_c = Counter(pairs)

In [15]:
top_pairs = pairs_c.most_common()

In [16]:
top_pairs

[(('United States', 'Spain'), 6028),
 (('United Kingdom', 'Spain'), 3883),
 (('France', 'Spain'), 3830),
 (('Germany', 'Spain'), 2575),
 (('Italy', 'Spain'), 2148),
 (('France', 'Argentina'), 1003),
 (('United States', 'Colombia'), 896),
 (('United States', 'Argentina'), 824),
 (('United States', 'Mexico'), 786),
 (('Argentina', 'Spain'), 782),
 (('España', 'Spain'), 719),
 (('United Kingdom', 'England'), 631),
 (('Spain', 'Colombia'), 599),
 (('France', 'Mexico'), 584),
 (('Belgium', 'Spain'), 532),
 (('Ireland', 'Spain'), 514),
 (('Switzerland', 'Spain'), 513),
 (('United Kingdom', 'United States'), 501),
 (('United States', 'New York (State)'), 488),
 (('Austria', 'Spain'), 475),
 (('Germany', 'Argentina'), 451),
 (('Canada', 'Spain'), 451),
 (('United Kingdom', 'Mexico'), 424),
 (('United Kingdom', 'Argentina'), 421),
 (('Mexico', 'Spain'), 419),
 (('Italy', 'Argentina'), 412),
 (('Poland', 'Spain'), 400),
 (('United States', 'England'), 397),
 (('Spain', 'Mexico'), 380),
 (('Argen

# Saving results

In [8]:
# autores_df.to_csv('autores6.csv', index=False)